In [2]:
!pip install pymilvus sentence_transformers

  Using cached grpcio-1.67.1-cp311-cp311-macosx_10_9_universal2.whl.metadata (3.9 kB)
  Using cached ujson-5.10.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (9.3 kB)
  Using cached milvus_lite-2.4.12-py3-none-macosx_11_0_arm64.whl.metadata (10.0 kB)
Using cached grpcio-1.67.1-cp311-cp311-macosx_10_9_universal2.whl (11.0 MB)
Using cached milvus_lite-2.4.12-py3-none-macosx_11_0_arm64.whl (17.4 MB)
Using cached ujson-5.10.0-cp311-cp311-macosx_11_0_arm64.whl (51 kB)
  Attempting uninstall: grpcio0m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [milvus-lite]
    Found existing installation: grpcio 1.71.0━━━━━━━━━━━━━━━━ 1/4 [milvus-lite]
    Uninstalling grpcio-1.71.0:0m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [milvus-lite]
      Successfully uninstalled grpcio-1.71.0━━━━━━━━━━━━━━━━━━ 1/4 [milvus-lite]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pymilvus]3/4 [pymilvus]


In [1]:
from sentence_transformers import SentenceTransformer
import torch

# 디바이스 설정: MPS 사용
device = "mps" if torch.backends.mps.is_available() else "cpu"
# device = "cpu"
# 모델 로드 및 디바이스 이동
model = SentenceTransformer("BAAI/bge-m3")
model.to(device)

/Users/ranifarm/.conda/envs/adaptive-rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SentenceTransformer(
  (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [2]:
from pymilvus import MilvusClient, DataType

# docker-compose에 설정한 19530포트에 연결
milvus_client = MilvusClient(uri="http://localhost:19530")
collection_name = "tragfile"

In [ ]:
# 스키마 정의 (사용안함)
# from pymilvus import CollectionSchema, FieldSchema, DataType

# milvus_client.create_collection(
#     collection_name=collection_name,
#     schema=CollectionSchema(fields=[
#         FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=False),
#         FieldSchema(name="file_name", dtype=DataType.VARCHAR, max_length=500),
#         FieldSchema(name="content", dtype=DataType.VARCHAR, max_length=10000),
#         FieldSchema(name="embed_vector", dtype=DataType.FLOAT_VECTOR, dim=1024),
#         FieldSchema(name="page_no", dtype=DataType.VARCHAR, max_length=100)
#         ]
#     )
# )

In [4]:
# 개선된 스키마
from pymilvus import CollectionSchema, FieldSchema, DataType

milvus_client.create_collection(
    collection_name="tragfile0102",
    schema=CollectionSchema(fields=[
        FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=False),
        FieldSchema(name="file_seq", dtype=DataType.INT64),
        FieldSchema(name="passage_seq", dtype=DataType.INT64),
        FieldSchema(name="file_name", dtype=DataType.VARCHAR, max_length=500),
        FieldSchema(name="content", dtype=DataType.VARCHAR, max_length=10000),
        FieldSchema(name="embed_vector", dtype=DataType.FLOAT_VECTOR, dim=1024),
        FieldSchema(name="page_no", dtype=DataType.VARCHAR, max_length=100)
    ])
)

In [5]:
import psycopg2, os
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

POSTGRE_NAME = os.getenv("POSTGRE_NAME")
POSTGRE_USER = os.getenv("POSTGRE_USER")
POSTGRE_HOST = os.getenv("POSTGRE_HOST")
POSTGRE_PORT = os.getenv("POSTGRE_PORT")
POSTGRE_PASSWORD = os.getenv("POSTGRE_PASSWORD")

# env value check
print(POSTGRE_NAME, POSTGRE_HOST)

mydb localhost


In [ ]:
# 기존에 만들어놓은 DB 테스트
conn = psycopg2.connect(
        dbname=POSTGRE_NAME,
        user=POSTGRE_USER,
        host=POSTGRE_HOST,
        port=POSTGRE_PORT,
        password=POSTGRE_PASSWORD
    )

try:
    # 비밀번호 없이 연결
    with conn.cursor() as cursor:        
        cursor = conn.cursor()
        cursor.execute("""
                       SELECT tf02.file_seq, tf02.passage_seq, tf01.file_nm, tf02.cont, tf02.page_no_chst
                       FROM tragfile0100 AS tf01
                       INNER JOIN tragfile0102 AS tf02
                       ON tf01.file_seq = tf02.file_seq;
                       """)
        
        result = cursor.fetchall()
        print(f"총 결과 행 수: {len(result)}")

        # 컬럼명 가져오기
        columns = [desc[0] for desc in cursor.description]

        # 데이터프레임 변환
        db_df = pd.DataFrame(result, columns=columns)


except Exception as e:
    print(f"Error: {e}")

finally:
    conn.close()
    print('db 연결 종료!')

db_df

In [7]:
# 메모리 관리 반영된 임베딩 코드
import torch
import gc
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

def sentence_to_vector(sentences: list, batch_size=4):
    device = "mps" if torch.backends.mps.is_available() else "cpu"
    model = SentenceTransformer("BAAI/bge-m3", device=device)
    model.eval()

    embeddings = []

    with torch.no_grad():
        for i in tqdm(range(0, len(sentences), batch_size), desc="Embedding"):
            batch = sentences[i:i + batch_size]

            # encode to numpy only, no GPU tensor retained
            emb = model.encode(batch, convert_to_tensor=False, show_progress_bar=False)
            embeddings.append(emb)

            # cleanup
            gc.collect()
            if torch.backends.mps.is_available():
                torch.mps.empty_cache()

    all_embeddings = np.vstack(embeddings)
    print(all_embeddings.shape)
    return all_embeddings

embedding_list = sentence_to_vector(list(db_df['cont']))
embedding_list[:5]

/Users/ranifarm/.conda/envs/adaptive-rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Embedding: 100%|██████████| 1517/1517 [51:58<00:00,  2.06s/it] 


(6066, 1024)


array([[-0.04364269,  0.00520862, -0.04609847, ..., -0.01214761,
        -0.00608617, -0.03677154],
       [-0.07635357,  0.0258653 , -0.04339358, ..., -0.03777843,
        -0.02506518, -0.02305122],
       [-0.04325062,  0.01397804, -0.04729412, ..., -0.02909242,
        -0.02145586, -0.02956823],
       [-0.03334042,  0.01946376, -0.03294627, ..., -0.0483857 ,
        -0.02976034, -0.03486133],
       [-0.02545958,  0.03871691, -0.03982378, ..., -0.01969385,
        -0.03979599, -0.02641861]], dtype=float32)

In [12]:
# insert 데이터 준비
id_list = range(len(db_df))
file_seq_list = list(db_df['file_seq'])
passage_seq_list = list(db_df['passage_seq'])
file_nm_list = list(db_df['file_nm'])
cont_list = list(db_df['cont'])
page_no_chst_list = list(db_df['page_no_chst'])
collection_name="tragfile0102"

data = [
    {"id": id, "file_seq": fsl, "passage_seq": psl, "file_name": fname, "content": cont, "embed_vector": emb, "page_no": pno}
    for id, fsl, psl, fname, cont, pno, emb in zip(id_list, file_seq_list, passage_seq_list, file_nm_list, cont_list, page_no_chst_list, embedding_list)
]

print(len(id_list), len(file_nm_list), len(cont_list), len(page_no_chst_list), len(embedding_list), len(data), len(file_seq_list), len(passage_seq_list), collection_name)

6066 6066 6066 6066 6066 6066 6066 6066 tragfile0102


set()

In [13]:
batch_size = 1000
for i in range(0, len(data), batch_size):
    chunk = data[i:i + batch_size]
    result = milvus_client.insert(collection_name=collection_name, data=chunk)
    print("Result:", result)  # 반환 객체 출력

Result: {'insert_count': 1000, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 2

In [17]:
# 다음과 같이 결과가 떨어짐.
# Result: {'insert_count': 1000, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,.. ,996, 997, 998, 999]}

In [14]:
index_params = MilvusClient.prepare_index_params()

# IndexParams 객체 생성
index_params.add_index(
    field_name="embed_vector",
    index_type="HNSW",
    index_name="vector_index",
    metric_type="COSINE",
    params={"M": 16, "efConstruction": 200}
)

# 인덱스 생성
try:
    milvus_client.create_index(
        collection_name=collection_name,
        index_params=index_params
    )
    print("Index created successfully")
except Exception as e:
    print(f"Error creating index: {e}")

# 컬렉션 로드
milvus_client.load_collection(collection_name=collection_name)
print("Collection loaded")

Index created successfully
Collection loaded


In [15]:
# 인덱스 
index_list = milvus_client.list_indexes(
    collection_name=collection_name,
    field_name="embed_vector"
)

print("Indexes on embed_vector:", index_list)

Indexes on embed_vector: ['vector_index']


In [16]:
res = milvus_client.describe_index(
    collection_name=collection_name,
    index_name="vector_index"
)

res

{'M': '16',
 'efConstruction': '200',
 'metric_type': 'COSINE',
 'index_type': 'HNSW',
 'field_name': 'embed_vector',
 'index_name': 'vector_index',
 'total_rows': 0,
 'indexed_rows': 0,
 'pending_index_rows': 0,
 'state': 'Finished'}

In [17]:
from pymilvus import connections, Collection

# Milvus에 연결 (Docker에서 기본 포트는 19530)
connections.connect(
    host="localhost",   # 도커가 localhost에서 열려 있다면
    port="19530"
)

# 이제 컬렉션 객체 생성 가능
collection = Collection(collection_name)

# 로드 후 데이터 건수 확인
collection.load()
print("총 데이터 건수:", collection.num_entities)

총 데이터 건수: 6066


In [43]:
from pymilvus import Collection

collection = Collection(collection_name)
collection.load()

# 첫 5개 엔티티 조회
results = collection.query(
    expr="",
    output_fields=["id"],  # 실제 필드명 넣기
    limit=7066
)

for r in results:
    print(r)

{'id': 0}
{'id': 1}
{'id': 2}
{'id': 3}
{'id': 4}
{'id': 5}
{'id': 6}
{'id': 7}
{'id': 8}
{'id': 9}
{'id': 10}
{'id': 11}
{'id': 12}
{'id': 13}
{'id': 14}
{'id': 15}
{'id': 16}
{'id': 17}
{'id': 18}
{'id': 19}
{'id': 20}
{'id': 21}
{'id': 22}
{'id': 23}
{'id': 24}
{'id': 25}
{'id': 26}
{'id': 27}
{'id': 28}
{'id': 29}
{'id': 30}
{'id': 31}
{'id': 32}
{'id': 33}
{'id': 34}
{'id': 35}
{'id': 36}
{'id': 37}
{'id': 38}
{'id': 39}
{'id': 40}
{'id': 41}
{'id': 42}
{'id': 43}
{'id': 44}
{'id': 45}
{'id': 46}
{'id': 47}
{'id': 48}
{'id': 49}
{'id': 50}
{'id': 51}
{'id': 52}
{'id': 53}
{'id': 54}
{'id': 55}
{'id': 56}
{'id': 57}
{'id': 58}
{'id': 59}
{'id': 60}
{'id': 61}
{'id': 62}
{'id': 63}
{'id': 64}
{'id': 65}
{'id': 66}
{'id': 67}
{'id': 68}
{'id': 69}
{'id': 70}
{'id': 71}
{'id': 72}
{'id': 73}
{'id': 74}
{'id': 75}
{'id': 76}
{'id': 77}
{'id': 78}
{'id': 79}
{'id': 80}
{'id': 81}
{'id': 82}
{'id': 83}
{'id': 84}
{'id': 85}
{'id': 86}
{'id': 87}
{'id': 88}
{'id': 89}
{'id': 90}
{'id': 91

In [ ]:
# (참고) 인덱스 제거할 때. release 해야 함.
milvus_client.release_collection(collection_name=collection_name)

milvus_client.drop_index(
    collection_name=collection_name,
    index_name="embed_vector"
)

In [50]:
from sentence_transformers import SentenceTransformer
from pymilvus import connections, Collection

# 1. 텍스트를 벡터로 바꾸기 (예: "내가 찾고 싶은 문장")
query_text = "최저임금 지금 얼마야"
model = SentenceTransformer("BAAI/bge-m3")  # HuggingFace 모델
query_vector = model.encode(query_text).tolist()

# 2. Milvus 연결
connections.connect(host="localhost", port="19530")
collection = Collection(collection_name)  # ← 실제 컬렉션 이름 넣기
collection.load()

# 3. 벡터로 검색
results = collection.search(
    data=[query_vector],
    anns_field="embed_vector",  # 임베딩 필드명
    param={"metric_type": "COSINE", "params": {"nprobe": 10}},
    limit=1,
    output_fields=["id", "file_name", "content", "page_no"]  # 보여주고 싶은 필드들
)

# nprobe 값을 늘리면?
# 	•	정확도↑: 더 많은 클러스터를 탐색하니 더 좋은 결과가 나올 수 있음
# 	•	속도↓: 더 많은 연산이 필요하므로 느려질 수 있음


# 4. 결과 출력
for hits in results:
    print(hits)
    print('='* 30)
    for hit in hits:
        print(f"ID: {hit.entity.get('id')}, Distance: {hit.distance:.4f}")
        print(f"Text: {hit.entity.get('content')}\n")

[{'id': 1508, 'distance': 0.6755048036575317, 'entity': {'id': 1508, 'file_name': '24년 최저임금제도 업무처리지침 (근로기준정책과-677 (2024.03.08. 시행)', 'content': '<붙임 2> 연도별 법정 최저임금액 현황(고시 기준)\n(단위: 원, %)\n∙ 적용년도: ʼ24\n∙ 시간급: 9,860\n∙ 일급: 78,880\n∙ 월환산액: 2,060,740\n∙ 인상률: 2.5\n\n∙ 적용년도: ʼ23\n∙ 시간급: 9,620\n∙ 일급: 76,960\n∙ 월환산액: 2,010,580\n∙ 인상률: 5.0\n\n∙ 적용년도: ʼ22\n∙ 시간급: 9,160\n∙ 일급: 73,280\n∙ 월환산액: 1,914,440\n∙ 인상률: 5.05\n\n∙ 적용년도: ʼ21\n∙ 시간급: 8,720\n∙ 일급: 69,760\n∙ 월환산액: 1,822,480\n∙ 인상률: 1.5\n\n∙ 적용년도: ʼ20\n∙ 시간급: 8,590\n∙ 일급: 68,720\n∙ 월환산액: 1,795,310\n∙ 인상률: 2.87\n\n∙ 적용년도: ʼ19\n∙ 시간급: 8,350\n∙ 일급: 66,800\n∙ 월환산액: 1,745,150\n∙ 인상률: 10.9\n\n∙ 적용년도: ʼ18\n∙ 시간급: 7,530\n∙ 일급: 60,240\n∙ 월환산액: 1,573,770\n∙ 인상률: 16.4\n\n∙ 적용년도: ʼ17\n∙ 시간급: 6,470\n∙ 일급: 51,760\n∙ 월환산액: 1,352,230\n∙ 인상률: 7.3\n\n∙ 적용년도: ʼ16\n∙ 시간급: 6,030\n∙ 일급: 48,240\n∙ 월환산액: 1,260,270\n∙ 인상률: 8.1\n\n∙ 적용년도: ʼ15\n∙ 시간급: 5,580\n∙ 일급: 44,640\n∙ 월환산액: 1,166,220\n∙ 인상률: 7.1\n\n∙ 적용년도: ʼ14\n∙ 시간급: 5,210\n∙ 일급: 41,680\n∙ 월환산액: 1,088,890\n∙ 인상률: 7

In [ ]:
# [{'id': 1508, 'distance': 0.6755048036575317, 'entity': {'id': 1508, 'content': '<붙임 2> 연도별 법정 최저임금액 현황(고시 기준)\n(단위: 원, %)\n∙ 적용년도: ʼ24\n∙ 시간급: 9,860\n∙ 일급: 78,880\n∙ 월환산액: 2,060,740\n∙ 인상률: 2.5\n\n∙ 적용년도: ʼ23\n∙ 시간급: 9,620\n∙ 일급: 76,960\n∙ 월환산액: 2,010,580\n∙ 인상률: 5.0\n\n∙ 적용년도: ʼ22\n∙ 시간급: 9,160\n∙ 일급: 73,280\n∙ 월환산액: 1,914,440\n∙ 인상률: 5.05\n\n∙ 적용년도: ʼ21\n∙ 시간급: 8,720\n∙ 일급: 69,760\n∙ 월환산액: 1,822,480\n∙ 인상률: 1.5\n\n∙ 적용년도: ʼ20\n∙ 시간급: 8,590\n∙ 일급: 68,720\n∙ 월환산액: 1,795,310\n∙ 인상률: 2.87\n\n∙ 적용년도: ʼ19\n∙ 시간급: 8,350\n∙ 일급: 66,800\n∙ 월환산액: 1,745,150\n∙ 인상률: 10.9\n\n∙ 적용년도: ʼ18\n∙ 시간급: 7,530\n∙ 일급: 60,240\n∙ 월환산액: 1,573,770\n∙ 인상률: 16.4\n\n∙ 적용년도: ʼ17\n∙ 시간급: 6,470\n∙ 일급: 51,760\n∙ 월환산액: 1,352,230\n∙ 인상률: 7.3\n\n∙ 적용년도: ʼ16\n∙ 시간급: 6,030\n∙ 일급: 48,240\n∙ 월환산액: 1,260,270\n∙ 인상률: 8.1\n\n∙ 적용년도: ʼ15\n∙ 시간급: 5,580\n∙ 일급: 44,640\n∙ 월환산액: 1,166,220\n∙ 인상률: 7.1\n\n∙ 적용년도: ʼ14\n∙ 시간급: 5,210\n∙ 일급: 41,680\n∙ 월환산액: 1,088,890\n∙ 인상률: 7.2\n\n∙ 적용년도: ʼ13\n∙ 시간급: 4,860\n∙ 일급: 38,880\n∙ 월환산액: 1,015,740\n∙ 인상률: 6.1\n\n∙ 적용년도: ʼ12\n∙ 시간급: 4,580\n∙ 일급: 36,640\n∙ 월환산액: 957,220\n∙ 인상률: 6.0\n\n∙ 적용년도: ʼ11\n∙ 시간급: 4,320\n∙ 일급: 34,560\n∙ 월환산액: 902,880\n∙ 인상률: 5.1\n\n∙ 적용년도: ʼ10\n∙ 시간급: 4,110\n∙ 일급: 32,880\n∙ 월환산액: 858,990\n∙ 인상률: 2.75\n\n∙ 적용년도: ʼ09\n∙ 시간급: 4,000\n∙ 일급: 32,000\n∙ 월환산액: 836,000\n∙ 인상률: 6.1\n\n∙ 적용년도: ʼ08\n∙ 시간급: 3,770\n∙ 일급: 30,160\n∙ 월환산액: 787,930\n∙ 인상률: 8.3\n\n∙ 적용년도: ʼ07\n∙ 시간급: 3,480\n∙ 일급: 27,840\n∙ 월환산액: 727,320\n∙ 인상률: 12.3\n\n∙ 적용년도: ʼ05.9.~06.12.\n∙ 시간급: 3,100\n∙ 일급: 24,800\n∙ 월환산액: 647,900\n∙ 인상률: 9.2\n\n∙ 적용년도: ʼ04.9.~05.8.\n∙ 시간급: 2,840\n∙ 일급: 22,720\n∙ 월환산액: 593,560\n∙ 인상률: 13.1\n\n∙ 적용년도: ʼ03.9.~04.8.\n∙ 시간급: 2,510\n∙ 일급: 20,080\n∙ 월환산액: 567,260\n∙ 인상률: 10.3\n\n∙ 적용년도: ʼ02.9.~03.8.\n∙ 시간급: 2,275\n∙ 일급: 18,200\n∙ 월환산액: 514,150\n∙ 인상률: 8.3\n\n∙ 적용년도: ʼ01.9.~02.8.\n∙ 시간급: 2,100\n∙ 일급: 16,800\n∙ 월환산액: 474,600\n∙ 인상률: 12.6\n\n∙ 적용년도: ʼ00.9.~01.8.\n∙ 시간급: 1,865\n∙ 일급: 14,920\n∙ 월환산액: 421,490\n∙ 인상률: 16.6\n\n∙ 적용년도: ʼ99.9.~00.8.\n∙ 시간급: 1,600\n∙ 일급: 12,800\n∙ 월환산액: 361,600\n∙ 인상률: 4.9\n\n∙ 적용년도: ʼ98.9.~99.8.\n∙ 시간급: 1,525\n∙ 일급: 12,200\n∙ 월환산액: 344,650\n∙ 인상률: 2.7\n\n∙ 적용년도: ʼ97.9.~98.8.\n∙ 시간급: 1,485\n∙ 일급: 11,880\n∙ 월환산액: 335,610\n∙ 인상률: 6.1\n\n∙ 적용년도: ʼ96.9.~97.8.\n∙ 시간급: 1,400\n∙ 일급: 11,200\n∙ 월환산액: 316,400\n∙ 인상률: 9.8\n\n∙ 적용년도: ʼ95.9.~96.8.\n∙ 시간급: 1,275\n∙ 일급: 10,200\n∙ 월환산액: 288,150\n∙ 인상률: 8.97\n\n∙ 적용년도: ʼ94.9.~95.8.\n∙ 시간급: 1,170\n∙ 일급: 9,360\n∙ 월환산액: 264,420\n∙ 인상률: 7.8\n\n∙ 적용년도: ʼ94.1.~8.\n∙ 시간급: 1,085\n∙ 일급: 8,680\n∙ 월환산액: 245,210\n∙ 인상률: 7.96\n\n∙ 적용년도: ʼ93\n∙ 시간급: 1,005\n∙ 일급: 8,040\n∙ 월환산액: 227,130\n∙ 인상률: 8.6\n\n∙ 적용년도: ʼ92\n∙ 시간급: 925\n∙ 일급: 7,400\n∙ 월환산액: 209,050\n∙ 인상률: 12.8\n\n∙ 적용년도: ʼ91\n∙ 시간급: 820\n∙ 일급: 6,560\n∙ 월환산액: 192,700\n∙ 인상률: 18.8\n\n∙ 적용년도: ʼ90\n∙ 시간급: 690\n∙ 일급: 5,520\n∙ 월환산액: 165,600\n∙ 인상률: 15.0\n\n∙ 적용년도: ʼ89\n∙ 시간급: 600\n∙ 일급: 4,800\n∙ 월환산액: 144,000\n∙ 인상률: 1그룹(29.7) / 2그룹(23.7)\n\n∙ 적용년도: ʼ88\n∙ 시간급: 1그룹(462.5) / 2그룹(487.5)\n∙ 일급: 1그룹(3,700) / 2그룹(3,900)\n∙ 월환산액: 1그룹(111,000) / 2그룹(117,000)\n∙ 인상률: -\n\n*1그룹 : 식료품, 섬유, 의복, 가죽, 신발, 나무, 종이, 고무, 플라스틱, 도기, 자기, 전기기기, 기타 제조업\n*2그룹 : 음료품, 담배, 가구, 인쇄출판, 산업화학, 기타화학, 석유정제, 석유석탄, 유리, 비금속, 철강, 비철\n금속, 조립금속, 기계, 운수장비, 정밀기계\n*월 환산 기준시간 : (88~90) 240 (91) 235 (92~04.8.) 226 (04.9.~) 209'}},

In [ ]:
t = {
    "success": "Y",
    "msg": "entered",
    "data": [
        {
            "type": "keyword",
            "file_name": "file",
            "content": "content",
            
            "metadata": {
                "cont_tsvector": {}
            }
        },
        {

        }
        
    ]

}